In [1]:
!python -V
import gc as garbage_colector

Python 3.10.4


# <center> SINCRONISMO - HILOS - PROCESOS - ASINCRONISMO</center>

<center>
    <img width="50%" lign="center" src="ejecuciones.png">
</center>

# <center> EJECUCION **SINCRONA** o **SERIAL**</center>

In [2]:
from datetime import datetime
from time import sleep
LIMITE = 11

def dormir_sincrono(tiempo):
    print('corriendo tarea de tiempo',tiempo)
    sleep(tiempo)
    return tiempo

In [3]:
%%time
datos = []

for tiempo_espera in range(1, LIMITE):
    datos.append(dormir_sincrono(tiempo_espera))

print(datos)
print(f'suma de valores de tiempo evaluados {sum(datos)}')

del datos
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo 3
corriendo tarea de tiempo 4
corriendo tarea de tiempo 5
corriendo tarea de tiempo 6
corriendo tarea de tiempo 7
corriendo tarea de tiempo 8
corriendo tarea de tiempo 9
corriendo tarea de tiempo 10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
suma de valores de tiempo evaluados 55
CPU times: user 21.7 ms, sys: 22 ms, total: 43.7 ms
Wall time: 55.1 s


(0, (700, 10, 10))

# <center> EJECUCION **ASINCRONA** Multitarea cooperativa</center>

In [4]:
import asyncio

# para .ipynb, event loop esta listo, usar await
# await main()

# para .py, usar asyncio.run
# asyncio.run(funcion_asincrona())

async def corrutina(tiempo:int) -> int:
    print('corriendo tarea de tiempo',tiempo)
    await asyncio.sleep(tiempo)
    return tiempo

datos = []
resul = []

inicia = datetime.now()

# crea el listado de corrutinas
for tiempo_espera in range(1, LIMITE):
    datos.append(asyncio.gather(corrutina(tiempo_espera)))
    
# recupera los valores de las corrutinas
for corrutina in datos:
    valor = await corrutina
    resul.append(valor[0])

finaliza = datetime.now()
print(f'Wall time: {finaliza.second - inicia.second} s') 
print(resul)

del datos, resul, inicia, finaliza
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo 3
corriendo tarea de tiempo 4
corriendo tarea de tiempo 5
corriendo tarea de tiempo 6
corriendo tarea de tiempo 7
corriendo tarea de tiempo 8
corriendo tarea de tiempo 9
corriendo tarea de tiempo 10
Wall time: 10 s
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


(0, (700, 10, 10))

# <center> EJECUCION **HILO** Subprocesamiento múltiple</center>
## <center> Este módulo construye interfaces de hilado de alto nivel sobre el módulo de más bajo nivel _thread. 

In [5]:
# libreria para procesamiento con hilos
from threading import Thread

# libreria que permite guardar los valores retornados en una cola
from queue import Queue as cola_classica

def almacenar_en_cola(f):
    def wrapper(*args):
        almacenar_resultados.put(f(*args))
    return wrapper

@almacenar_en_cola
def dormir_sincrono(tiempo):
    print('corriendo tarea de tiempo',tiempo)
    sleep(tiempo)
    return tiempo

def obtener_resultado_cola_tarea(cola):
    
    valores = []
    for indice_tarea in range(cola.qsize()):
        valores.append(cola.get(indice_tarea))
    
    return valores

In [6]:
%%time

# instancia de la cola para adicionar los resultados
almacenar_resultados = cola_classica()

# almacena las instancias generadas
listado_instancias = []   
    
# genera la instancias necesarias para ejecutar, las inicializa y las guarda en la lista
for tiempo_espera in range(1, LIMITE):
  
    instancia = Thread(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(tiempo_espera,))

    instancia.start() # Comienza la actividad del hilo. Esto debe llamarse como máximo una vez por objeto de hilo.
    
    listado_instancias.append(instancia)

# recorre la instancias generadass en la lista de instancias y bloque las ejecuciones hasta que termine    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join() # Espera a que salgan los hilos de trabajo. Espera a que el hilo termine. Esto bloquea el hilo llamador hasta que el hilo cuyo método join() es llamado finalice.

# ejecuta la funcion que se encarga de recuperar los resultados de las ejecuciones        
obtener_resultado_cola_tarea(almacenar_resultados)

# elimina las variables creaadas para la ejecucion
del almacenar_resultados, listado_instancias

# muestra la cantidad de objetos que se mantienen despues de la ejecucion  y el espacio ocupado por estos
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo 3
corriendo tarea de tiempo 4
corriendo tarea de tiempo 5
corriendo tarea de tiempo 6
corriendo tarea de tiempo 7
corriendo tarea de tiempo 8
corriendo tarea de tiempo 9
corriendo tarea de tiempo 10
CPU times: user 25.3 ms, sys: 0 ns, total: 25.3 ms
Wall time: 10 s


(0, (700, 10, 10))

## <center> INSTANCIAS CON **LIST COMPREHENSION**

In [7]:
%%time

# instancia de la cola para adicionar los resultados
almacenar_resultados = cola_classica()

# almacena las instancias generadas
listado_instancias = []   

# genera la instancias necesarias para ejecutar, las inicializa y las guarda en la lista
instancias =  [Thread(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(tiempo_espera,)) for tiempo_espera in range(1, LIMITE)]
    
# recorre cada instancia, la inicializa y luego la guarda en la lista de instancias
for instancia in instancias:

    instancia.start() # Comienza la actividad del hilo. Esto debe llamarse como máximo una vez por objeto de hilo.
    
    listado_instancias.append(instancia)

# recorre la instancias generadass en la lista de instancias y bloque las ejecuciones hasta que termine    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join() # Espera a que salgan los hilos de trabajo. Espera a que el hilo termine. Esto bloquea el hilo llamador hasta que el hilo cuyo método join() es llamado finalice.

# ejecuta la funcion que se encarga de recuperar los resultados de las ejecuciones        
obtener_resultado_cola_tarea(almacenar_resultados)

# elimina las variables creaadas para la ejecucion
del almacenar_resultados, listado_instancias

# muestra la cantidad de objetos que se mantienen despues de la ejecucion  y el espacio ocupado por estos
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo 3
corriendo tarea de tiempo 4
corriendo tarea de tiempo 5
corriendo tarea de tiempo 6
corriendo tarea de tiempo 7
corriendo tarea de tiempo 8
corriendo tarea de tiempo 9
corriendo tarea de tiempo 10
CPU times: user 26.3 ms, sys: 483 µs, total: 26.8 ms
Wall time: 10 s


(0, (700, 10, 10))

# <center> EJECUCION **PROCESO MULTIPLE**</center>
## <center> Este módulo permite crear procesos (spawning) utilizando una API similar al módulo threading. ofrece concurrencia tanto local como remota. permite aprovechar al máximo múltiples procesadores en una máquina determinada. 

In [8]:
# libreria que permite usar la computacion paralela
from multiprocessing import Process

# libreria que permite guardar los valores retornados en una cola
from multiprocessing import Queue as cola_proceso

def almacenar_en_cola_proceso(f):
    def wrapper(*args):
        almacenar_resultados_proceso.put(f(*args))
    return wrapper

@almacenar_en_cola_proceso
def dormir_sincrono(tiempo):
    print('corriendo tarea de tiempo',tiempo)
    sleep(tiempo)
    return tiempo

def obtener_resultado_cola_tarea_while(cola):
    
    valores = []
    while not cola.empty():
        result = cola.get()
        valores.append(result)    
    return valores

In [9]:
%%time

# instancia de la cola para adicionar los resultados
almacenar_resultados_proceso = cola_proceso()

# almacena las instancias generadas
listado_instancias = []

for tiempo_espera in range(1, LIMITE):
  
    instancia = Process(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(tiempo_espera,))
    instancia.start() # Comienza la actividad del proceso. Esto debe llamarse como máximo una vez por objeto de proceso.  
    listado_instancias.append(instancia)

for tipo_instancia in listado_instancias:        
        tipo_instancia.join() # Espera a que salgan los procesos de trabajo. Se debe llamar close() o terminate() antes de usar join().

obtener_resultado_cola_tarea_while(almacenar_resultados_proceso)

del almacenar_resultados_proceso, listado_instancias
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo 1corriendo tarea de tiempocorriendo tarea de tiempo 
2 corriendo tarea de tiempo corriendo tarea de tiempo34


corriendo tarea de tiempo  5corriendo tarea de tiempocorriendo tarea de tiempo
6 
 corriendo tarea de tiempo78corriendo tarea de tiempo 

 109

CPU times: user 62 ms, sys: 112 ms, total: 174 ms
Wall time: 10.2 s


(0, (700, 10, 10))

# <center> TOTAL PROCESADORES

In [10]:
import psutil
total_cpu = psutil.cpu_count()
mitad_procesamiento = total_cpu//2
total_cpu, mitad_procesamiento

(4, 2)

# <center> EJECUCION **THREAD POOL DE MULTIPROCESO**</center>

In [11]:
def dormir_sincrono_multiprocessing(tiempo):
    print('corriendo tarea de tiempo ',tiempo)
    sleep(tiempo)
    return tiempo

# <center> **PROCESADORES POR DEFECTO**</center>

In [12]:
# libreria quenpermite usar multihilos con la libreria multiprocessing
from multiprocessing.pool import ThreadPool

In [13]:
%%time

pool = ThreadPool()
multihilo = pool.map(dormir_sincrono_multiprocessing, range(1, LIMITE))
pool.close()
pool.join()

print(multihilo)

del pool, multihilo
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo corriendo tarea de tiempo  2
 1
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 37.6 ms, sys: 1.99 ms, total: 39.6 ms
Wall time: 18 s


(0, (700, 10, 10))

# <center> **PROCESADORES ASIGNADOS** processes</center>

In [14]:
%%time

with ThreadPool(processes=mitad_procesamiento) as pool_de_hilos:
    multihilo = pool_de_hilos.map(dormir_sincrono_multiprocessing, range(1, LIMITE))

print(multihilo)

del pool_de_hilos, multihilo
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  3
corriendo tarea de tiempo  2
corriendo tarea de tiempo corriendo tarea de tiempo  4
 5
corriendo tarea de tiempo  7
corriendo tarea de tiempo  6
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 36 ms, sys: 1.38 ms, total: 37.4 ms
Wall time: 33.1 s


(0, (700, 10, 10))

# <center> EJECUCION **POOL DE MULTIPROCESO**</center>

# <center> **PROCESADORES POR DEFECTO**</center>

In [15]:
# libreria quenpermite usar multiprocesos con la libreria multiprocessing
from multiprocessing.pool import Pool

In [16]:
%%time

pool = Pool()
multiproceso = pool.map(dormir_sincrono_multiprocessing, range(1, LIMITE))
pool.close() # Impide que se envíen más tareas a la piscina (pool). Una vez que se hayan completado todas las tareas, se cerrarán los procesos de trabajo.
pool.join() # Espera a que salgan los procesos de trabajo. Se debe llamar close() o terminate() antes de usar join().

print(multiproceso)

del pool, multiproceso
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo corriendo tarea de tiempo corriendo tarea de tiempo  corriendo tarea de tiempo  3  
42
1

corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 84.7 ms, sys: 7.84 ms, total: 92.6 ms
Wall time: 18.1 s


(0, (700, 10, 10))

## <center> **PROCESOS ADMINISTRADOR DE CONTEXTO**</center>

In [17]:
%%time
with Pool() as pool_de_procesos:
    multiproceso = pool_de_procesos.map(dormir_sincrono_multiprocessing, range(1, LIMITE))

print(multiproceso)

del pool_de_procesos, multiproceso
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo corriendo tarea de tiempo corriendo tarea de tiempo corriendo tarea de tiempo    21 

4
3
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 64.3 ms, sys: 34.7 ms, total: 99 ms
Wall time: 18.1 s


(0, (700, 10, 10))

# <center> **PROCESADORES ASIGNADOS** processes</center>

In [18]:
%%time
with Pool(processes=mitad_procesamiento) as pool_de_procesos:
    multiproceso = pool_de_procesos.map(dormir_sincrono_multiprocessing, range(1, LIMITE))
    
print(multiproceso)

del pool_de_procesos, multiproceso
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo corriendo tarea de tiempo   3
1
corriendo tarea de tiempo  2
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  7
corriendo tarea de tiempo  6
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 82.3 ms, sys: 2.39 ms, total: 84.7 ms
Wall time: 33.1 s


(0, (700, 10, 10))

# <center> EJECUCION **THREAD POOL EXECUTOR DE CONCURRENT.FUTURES**</center>
## <center> Este módulo provee una interfaz de alto nivel para ejecutar invocables de forma **asincrónica**. Usando **ThreadPoolExecutor, o ProcessPoolExecutor**. Ambos implementan la misma interfaz definida por la **clase abstracta Executor** compatible con asyncio.

In [19]:
def dormir_concurrent_futures(tiempo):
    print('corriendo tarea de tiempo ',tiempo)
    sleep(tiempo)
    return tiempo

# <center> **PROCESADORES POR DEFECTO**</center>

In [20]:
# importa librerias para trabajar concurrencia
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

In [21]:
%%time

executor = ThreadPoolExecutor()

futures = []
resultado = []
for tiempo in range(1, LIMITE):
    future = executor.submit(dormir_concurrent_futures, tiempo)
    futures.append(future)

for future in as_completed(futures):
    resultado.append(future.result())

executor.shutdown()
print(resultado)

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  2
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 29.6 ms, sys: 2.35 ms, total: 31.9 ms
Wall time: 12 s


(0, (700, 10, 10))

# <center> **EL ADMINISTRADOR DE CONTEXTO** NO GARANTIZA EL ORDEN</center>

In [22]:
%%time

futures = []
resultado = []

with ThreadPoolExecutor() as executor:
    for tiempo in range(1, LIMITE):
        future = executor.submit(dormir_concurrent_futures, tiempo)
        futures.append(future)    

    for future in as_completed(futures):
        resultado.append(future.result())
    
print(resultado)

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  2
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 15.3 ms, sys: 16.5 ms, total: 31.8 ms
Wall time: 12 s


(0, (700, 10, 10))

# <center> **PROCESADORES ASIGNADOS ADMINISTRADOR DE CONTEXTO** max_workers</center>

In [23]:
%%time

futures = []
resultado = []

with ThreadPoolExecutor(max_workers=mitad_procesamiento) as executor:
    for tiempo in range(1, LIMITE):
        future = executor.submit(dormir_concurrent_futures, tiempo)
        futures.append(future)    

    for future in as_completed(futures):
        resultado.append(future.result())
    
print(resultado)

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  2
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 37.4 ms, sys: 1.68 ms, total: 39.1 ms
Wall time: 30.1 s


(0, (700, 10, 10))

# <center> **PROCESADORES ASIGNADOS MAP** MANTIENE EL ORDEN DE LA INFORMACION</center>

In [24]:
%%time

with ThreadPoolExecutor(max_workers=mitad_procesamiento) as executor:
    futures = executor.map(dormir_concurrent_futures, range(1, LIMITE))
list(futures) 

del executor, futures
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  2
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
CPU times: user 33.2 ms, sys: 396 µs, total: 33.6 ms
Wall time: 30.1 s


(0, (700, 10, 10))

# <center> EJECUCION **PROCESS POOL EXECUTOR DE CONCURRENT.FUTURES**</center>

In [25]:
# importa librerias para trabajar concurrencia
from concurrent.futures import ProcessPoolExecutor

# <center> **PROCESADORES POR DEFECTO**</center>

In [26]:
%%time

executor = ProcessPoolExecutor()

futures = []
resultado = []
for tiempo in range(1, LIMITE):
    future = executor.submit(dormir_concurrent_futures, tiempo) # submit -> Programa el invocable, fn, para que se ejecute como fn(*args, **kwargs) y devuelve un objeto Future que representa la ejecución del invocable.
    futures.append(future)

# as_completed -> espera que se complete cada llamado de future para poder mostrar su resultado.
# usar as_completed -> cuando se una el metodo submit para correr una funcion
for future in as_completed(futures):
    resultado.append(future.result()) # .result() obtiene el resultado de ejecutor

executor.shutdown() # Indica al ejecutor que debe liberar todos los recursos que está utilizando cuando los futuros actualmente pendientes de ejecución finalicen.

print(resultado)

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1corriendo tarea de tiempo 
 corriendo tarea de tiempo 
 23corriendo tarea de tiempo 
 4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 78.4 ms, sys: 30.9 ms, total: 109 ms
Wall time: 18.1 s


(0, (700, 10, 10))

# <center> **EL ADMINISTRADOR DE CONTEXTO** NO GARANTIZA EL ORDEN</center>

In [27]:
%%time

futures = []
resultado = []

with ProcessPoolExecutor() as executor:
    for tiempo in range(1, LIMITE):
        future = executor.submit(dormir_concurrent_futures, tiempo)
        futures.append(future)    

    for future in as_completed(futures):
        resultado.append(future.result())
    
print(resultado)

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo   corriendo tarea de tiempo 12
corriendo tarea de tiempo 
corriendo tarea de tiempo  3 4

corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 64.7 ms, sys: 52.4 ms, total: 117 ms
Wall time: 18.1 s


(0, (700, 10, 10))

# <center> **PROCESADORES POR DEFECTO USANDO MAP**</center>

In [28]:
%%time

with ProcessPoolExecutor() as executor:
    futures = executor.map(dormir_concurrent_futures, range(1, LIMITE))
list(futures)

del executor, futures
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  corriendo tarea de tiempo  1
2
corriendo tarea de tiempo corriendo tarea de tiempo  4 
3
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
CPU times: user 55.4 ms, sys: 54.7 ms, total: 110 ms
Wall time: 18.1 s


(0, (700, 10, 10))

Jocelyn Arellano# <center> **PROCESADORES ASIGNADOS** max_workers</center>

In [29]:
%%time

futures = []
resultado = []
with ProcessPoolExecutor(max_workers=mitad_procesamiento) as executor:    
    futures = executor.map(dormir_concurrent_futures, range(1, LIMITE))
list(futures)  

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo corriendo tarea de tiempo   2
1
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
CPU times: user 37.5 ms, sys: 52.3 ms, total: 89.7 ms
Wall time: 30.1 s


(0, (700, 10, 10))

# *****************************************************************************************************************************************************************************************************************************

# <center> HILOS I/O VS PROCESOS I/O</center>

In [30]:
cantidad = 100_000

def lectura_io(value):
    with open('demofile.txt', 'rb') as f:
        return f.read(value * 100)

## <center> HILOS I/O</center>

### <center> LIMITAR LA CANTIDAD DE TAREAS PARA NO DESBORDAR LA MEMORIA POR ALMACENAMIENTO DE RESULTADOS EN LA COLA</center>

In [31]:
%%time
almacenar_resultados = cola_classica()
listado_instancias = [] 

@almacenar_en_cola
def lectura_io_hilo(value):
    with open('demofile.txt', 'rb') as f:
        return f.read(value * 100)
    
for valor_lectura in range(1, cantidad//2):
  
    instancia = Thread(name=f'ejecucion de tarea # {valor_lectura}', target=lectura_io_hilo, args=(valor_lectura,))
    instancia.start()    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()
        
resultado = obtener_resultado_cola_tarea(almacenar_resultados)
print(resultado[:100])

del almacenar_resultados, listado_instancias, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

[b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge

(0, (700, 10, 10))

### <center> CAPACIDAD DE PROCESAR TODAS LAS TAREAS SIN DESBORDAR LA MEMORIA</center>

In [32]:
%%time

with ThreadPool() as multi_hilos:        
    resultado = multi_hilos.map(lectura_io, range(cantidad))
    print(len(resultado))
    
print(list(resultado)[:100])

del multi_hilos, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

100000
[b'', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronis

(0, (700, 10, 10))

In [33]:
%%time

with ThreadPoolExecutor() as multi_hilos:        
    resultado = multi_hilos.map(lectura_io, range(cantidad))
    print(len(list(resultado)))
    
print(list(resultado))

del multi_hilos, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

100000
[]
CPU times: user 17.8 s, sys: 51.7 s, total: 1min 9s
Wall time: 30.2 s


(0, (700, 10, 10))

## <center> PROCESOS I/O</center>

In [34]:
%%time
almacenar_resultados_proceso = cola_proceso()
listado_instancias = [] 

@almacenar_en_cola_proceso
def lectura_io_proceso(value):
    with open('demofile.txt', 'rb') as f:
        return f.read(value * 100)
    
for valor_lectura in range(1, cantidad//200):
  
    instancia = Process(name=f'ejecucion de tarea # {valor_lectura}', target=lectura_io_proceso, args=(valor_lectura,))
    instancia.start()    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()
        
resultado = obtener_resultado_cola_tarea_while(almacenar_resultados_proceso)
print(resultado[:100])

del almacenar_resultados_proceso, listado_instancias, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

[b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge

(0, (700, 10, 10))

In [35]:
%%time
with Pool() as multiprocessing:
    resultado = multiprocessing.map(lectura_io, range(cantidad))
    print(len(list(resultado)))
    
print(list(resultado)[:100])

del multiprocessing, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

100000
[b'', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronis

(0, (700, 10, 10))

In [36]:
%%time
with ProcessPoolExecutor() as concurrent_futures:
    resultado = concurrent_futures.map(lectura_io, range(cantidad))
    
print(list(resultado)[:100])

del concurrent_futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

[b'', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'

(0, (700, 10, 10))

# <center> HILOS CPU VS PROCESOS CPU</center>

In [37]:
import math

inicio = 1_099_726_899_294_999
fin    = 1_099_726_899_299_999

PRIMES = range(inicio, fin) 

def is_prime(n):
    if n < 2 or n % 2 == 0:
        return False
    if n == 2:
        return True
    
    sqrt_n = int(math.floor(math.sqrt(n)))
    
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

print(f'total iteraciones {fin - inicio}')

total iteraciones 5000


## <center> SINCRONO CPU</center>

In [38]:
%%time

print(list(map(is_prime, PRIMES)))

[False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False

## <center> HILOS CPU</center>

In [39]:
%%time
almacenar_resultados = cola_classica()
listado_instancias = [] 

@almacenar_en_cola
def is_prime_hilo(n):
    if n < 2 or n % 2 == 0:
        return False
    if n == 2:
        return True
    
    sqrt_n = int(math.floor(math.sqrt(n)))
    
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True
    
for valor_lectura in PRIMES:
  
    instancia = Thread(name=f'ejecucion de tarea # {valor_lectura}', target=is_prime_hilo, args=(valor_lectura,))
    instancia.start()    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()
        
resultado = obtener_resultado_cola_tarea(almacenar_resultados)
print(resultado[:100])

del almacenar_resultados, listado_instancias, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
CPU times: user 5min 1s, sys: 0 ns, total: 5min 1s
Wall time: 4min 48s


(0, (700, 10, 10))

In [40]:
%%time
resultado = []

with ThreadPool() as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):       
        resultado.append((number, prime))

print('cantidad de valores procesados', len(resultado))

del executor, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

cantidad de valores procesados 5000
CPU times: user 4min 46s, sys: 0 ns, total: 4min 46s
Wall time: 4min 40s


(0, (700, 10, 10))

In [41]:
%%time
resultado = []

with ThreadPoolExecutor() as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):       
        resultado.append((number, prime))

print('cantidad de valores procesados', len(resultado))
        
del executor, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

cantidad de valores procesados 5000
CPU times: user 5min 20s, sys: 0 ns, total: 5min 20s
Wall time: 5min 4s


(0, (700, 10, 10))

## <center> PROCESOS CPU</center>

In [42]:
%%time
almacenar_resultados_proceso = cola_proceso()
listado_instancias = [] 

@almacenar_en_cola_proceso
def is_prime_proceso(n):
    if n < 2 or n % 2 == 0:
        return False
    if n == 2:
        return True
    
    sqrt_n = int(math.floor(math.sqrt(n)))
    
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True
    
for valor_lectura in PRIMES:
  
    instancia = Process(name=f'ejecucion de tarea # {valor_lectura}', target=is_prime_proceso, args=(valor_lectura,))
    instancia.start()    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()

print('cantidad de valores procesados', len(obtener_resultado_cola_tarea_while(almacenar_resultados_proceso)))

del almacenar_resultados_proceso, listado_instancias
garbage_colector.collect(), garbage_colector.get_threshold()

cantidad de valores procesados 5000
CPU times: user 37.6 s, sys: 0 ns, total: 37.6 s
Wall time: 3min 39s


(0, (700, 10, 10))

In [43]:
%%time

resultado = []
with Pool() as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):       
        resultado.append((number, prime))

print('cantidad de valores procesados', len(resultado))

del executor, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

cantidad de valores procesados 5000
CPU times: user 217 ms, sys: 0 ns, total: 217 ms
Wall time: 2min 55s


(0, (700, 10, 10))

In [44]:
%%time

resultado = []
with ProcessPoolExecutor() as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):       
        resultado.append((number, prime))

print('cantidad de valores procesados', len(resultado))
        
del executor, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

cantidad de valores procesados 5000
CPU times: user 1.94 s, sys: 0 ns, total: 1.94 s
Wall time: 2min 50s


(0, (700, 10, 10))

## <center> NUMBA CPU</center>

In [45]:
#pip install numba # Numba es el paquete compilador, esto depende de llvmlite.
#pip install llvmlite # llvmlite es un paquete de enlace ligero para las API de LLVM, depende de LLVM, LLVM es el marco del compilador JIT para producir código ejecutable a partir de varias entradas.
!pip show numba

Name: numba
Version: 0.55.1
Summary: compiling Python code using LLVM
Home-page: https://numba.pydata.org
Author: 
Author-email: 
License: BSD
Location: /usr/local/lib/python3.10/site-packages
Requires: llvmlite, numpy, setuptools
Required-by: 


In [46]:
from numba import jit

@jit(nopython=True)
def is_prime_numba(n):
    if n < 2 or n % 2 == 0:
        return False
    if n == 2:
        return True
    
    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

In [47]:
%%time
print(f'total iteraciones {fin - inicio}')
print(list(map(is_prime_numba, PRIMES)))

garbage_colector.collect(), garbage_colector.get_threshold()

total iteraciones 5000
[False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, Tr

(3644, (700, 10, 10))

In [48]:
%%time

inicio = 1_099_726_899_249_999
fin    = 1_099_726_899_299_999

PRIMES = range(inicio, fin) 

print(f'total iteraciones {fin - inicio}')
print(list(map(is_prime_numba, PRIMES)))
garbage_colector.collect(), garbage_colector.get_threshold()

total iteraciones 50000
[False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, Fals

(0, (700, 10, 10))

# *****************************************************************************************************************************************************************************************************************************

# <center> EJECUCION 1800 TAREAS DE 2 SEGUNDOS, TOMARIA 1 HORA EN EJECUCION SINCRONA</center>

In [49]:
import random
limite_tareas = 1800

## <center> ASINCRONO</center>

In [50]:
async def corrutina(tiempo:int) -> int:    
    await asyncio.sleep(tiempo)
    return random.randint(0, limite_tareas)

inicia = datetime.now()
lista = [corrutina(2) for i in range(limite_tareas)]

resultado = await asyncio.gather(*lista)

finaliza = datetime.now()
print(len(resultado))
print(f'Wall time: {finaliza.second - inicia.second} s')
print(resultado)

del lista, resultado, inicia, finaliza
garbage_colector.collect(), garbage_colector.get_threshold()

1800
Wall time: 2 s
[1658, 1411, 788, 1245, 1674, 531, 1081, 1327, 1427, 730, 1770, 1762, 1585, 1616, 1001, 1757, 1783, 377, 504, 332, 152, 353, 442, 1093, 988, 426, 235, 507, 1338, 935, 1166, 1114, 1554, 762, 1734, 870, 1585, 1029, 631, 321, 377, 398, 481, 322, 1194, 9, 843, 738, 1201, 725, 466, 190, 481, 1018, 1525, 920, 522, 1276, 1294, 1359, 471, 333, 600, 121, 1560, 668, 1410, 714, 496, 1456, 764, 804, 498, 850, 1197, 917, 1210, 984, 1298, 927, 1357, 1693, 1251, 1285, 110, 1725, 347, 408, 1475, 1607, 162, 51, 943, 1713, 377, 1583, 1377, 1194, 1145, 793, 1064, 1437, 19, 430, 413, 1456, 1215, 361, 1463, 1636, 1109, 1125, 930, 1290, 530, 1507, 1093, 902, 1260, 996, 1122, 552, 1544, 1753, 604, 685, 782, 24, 1396, 124, 1477, 324, 1067, 871, 677, 419, 1176, 805, 1130, 1697, 1705, 617, 1412, 363, 761, 1120, 1671, 1560, 1294, 1464, 1200, 308, 1364, 1742, 1226, 1050, 675, 516, 1632, 961, 1203, 246, 1177, 170, 816, 1439, 1514, 1084, 1515, 879, 81, 1570, 504, 1248, 90, 1403, 1115, 568, 282, 

(0, (700, 10, 10))

## <center> THREAD</center>

In [51]:
%%time

listado_instancias = []
# instancia de la cola para adicionar los resultados
almacenar_resultados = cola_classica()


@almacenar_en_cola
def dormir_sincrono(tiempo:int) -> int:    
    sleep(tiempo)
    return random.randint(0,limite_tareas)

    
for tiempo_espera in range(limite_tareas):
  
    instancia = Thread(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(2,))

    instancia.start()
    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()

resultado = obtener_resultado_cola_tarea(almacenar_resultados)
print(len(resultado))
print(resultado)

del listado_instancias, almacenar_resultados, instancia, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

1800
[1733, 1061, 972, 1122, 116, 232, 386, 1610, 1339, 1259, 220, 1400, 38, 640, 662, 568, 1589, 491, 1023, 906, 107, 1444, 539, 1417, 200, 1037, 963, 1723, 428, 700, 767, 1694, 1736, 1716, 1797, 827, 991, 713, 1257, 1767, 802, 776, 744, 467, 659, 1332, 868, 64, 1703, 668, 352, 662, 1565, 1497, 820, 678, 1533, 93, 1264, 118, 1195, 643, 1090, 395, 421, 1062, 1682, 663, 339, 1485, 1397, 336, 380, 1753, 551, 253, 1614, 1263, 748, 1424, 550, 111, 1781, 1344, 854, 905, 251, 1780, 1401, 904, 465, 1435, 910, 232, 456, 864, 251, 1428, 729, 1443, 287, 1079, 1787, 1324, 548, 1252, 1264, 875, 105, 817, 126, 1295, 1751, 494, 824, 1580, 44, 1513, 907, 1278, 1747, 542, 1361, 1621, 520, 1620, 377, 1314, 467, 1086, 106, 1319, 1386, 496, 284, 1265, 1739, 1270, 948, 114, 1475, 1758, 541, 1398, 173, 180, 112, 354, 575, 317, 1128, 261, 1262, 62, 1711, 1182, 60, 773, 639, 1267, 503, 553, 714, 999, 1721, 751, 1641, 599, 145, 1235, 412, 1158, 1669, 593, 999, 159, 1317, 1553, 344, 1342, 89, 437, 888, 1038, 1

(0, (700, 10, 10))

## <center> PROCESS</center>

In [52]:
%%time

# instancia de la cola para adicionar los resultados
listado_instancias = []

# instancia de la cola para adicionar los resultados
almacenar_resultados_proceso = cola_proceso()

@almacenar_en_cola_proceso
def dormir_sincrono(tiempo:int) -> int:    
    sleep(tiempo)
    return random.randint(0,limite_tareas)
        

for tiempo_espera in range(limite_tareas):
  
    instancia = Process(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(2,))
    instancia.start()
    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()

resultado = obtener_resultado_cola_tarea_while(almacenar_resultados_proceso)
print(len(resultado))
print(resultado)

del listado_instancias, almacenar_resultados_proceso, instancia, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

1800
[1261, 386, 843, 422, 1071, 647, 606, 264, 48, 1395, 1399, 500, 1085, 656, 615, 933, 1334, 867, 1365, 396, 1746, 1134, 20, 982, 1097, 749, 1055, 1197, 937, 607, 572, 1774, 584, 222, 213, 1313, 1192, 455, 829, 1101, 1103, 1577, 714, 883, 762, 127, 1571, 607, 1559, 683, 633, 383, 1414, 744, 1396, 722, 1086, 1786, 1412, 56, 1120, 1442, 1765, 174, 544, 549, 1719, 1222, 1736, 65, 581, 1551, 174, 803, 632, 992, 1475, 1307, 228, 583, 198, 171, 52, 596, 387, 28, 67, 836, 292, 1561, 753, 1256, 738, 109, 349, 1387, 1440, 69, 1124, 244, 1114, 725, 1747, 1081, 682, 1514, 1746, 942, 1790, 992, 1561, 877, 1410, 261, 1270, 604, 1055, 806, 233, 1342, 755, 1662, 1092, 267, 1760, 652, 1029, 1411, 402, 1138, 1731, 528, 529, 1148, 212, 791, 221, 1617, 1335, 1797, 1462, 609, 1592, 10, 1475, 1025, 878, 477, 152, 378, 237, 1372, 477, 1391, 18, 1568, 1569, 906, 491, 929, 1730, 1529, 1630, 67, 1143, 1350, 20, 1175, 1491, 603, 620, 666, 1484, 243, 674, 1486, 195, 976, 1499, 290, 1666, 327, 981, 578, 27, 56

(0, (700, 10, 10))

## <center> **POOL PROCESS DE MULTIPROCESO**</center>

In [53]:
%%time

def dormir_sincrono_multiprocessing(contador:int) -> int:    
    sleep(2)
    return random.randint(0,limite_tareas)

with Pool() as multiprocesing:
    pool = multiprocesing.map(dormir_sincrono_multiprocessing, range(limite_tareas//3))
print(list(pool))

del multiprocesing, pool
garbage_colector.collect(), garbage_colector.get_threshold()

[353, 1216, 1705, 143, 1327, 326, 1492, 415, 1066, 803, 373, 1376, 658, 1575, 444, 427, 1545, 540, 1469, 702, 611, 1156, 697, 398, 1328, 80, 644, 544, 1615, 1495, 884, 646, 721, 1006, 750, 338, 739, 939, 1331, 1098, 731, 1706, 1609, 753, 957, 1756, 912, 1432, 917, 1795, 734, 517, 630, 1691, 706, 1502, 1505, 1794, 1465, 1046, 776, 233, 818, 768, 766, 1652, 1222, 698, 126, 47, 905, 804, 801, 1080, 233, 770, 429, 1455, 1432, 416, 1202, 1112, 0, 1450, 413, 239, 1204, 955, 354, 609, 1705, 164, 1321, 1033, 1591, 351, 475, 689, 1706, 1420, 1311, 616, 1142, 1561, 1395, 945, 761, 1455, 37, 1238, 519, 1658, 232, 530, 1592, 11, 1134, 1787, 210, 245, 1627, 999, 1456, 1325, 827, 1788, 1718, 1122, 1106, 1691, 609, 1556, 365, 10, 1662, 780, 1095, 1421, 375, 1485, 718, 1225, 1709, 462, 1037, 74, 151, 1494, 494, 931, 1079, 893, 1109, 1373, 155, 993, 1043, 1443, 146, 781, 1069, 1432, 1511, 1483, 420, 561, 1768, 607, 268, 1777, 759, 1022, 1655, 1091, 1262, 32, 619, 214, 391, 479, 1097, 1799, 859, 1050, 9

(0, (700, 10, 10))

## <center> **THREAD POOL DE MULTIPROCESO**</center>

In [54]:
%%time

with ThreadPool() as multiprocesing:
    pool = multiprocesing.map(dormir_sincrono_multiprocessing, range(limite_tareas//3))
print(list(pool))

del multiprocesing, pool
garbage_colector.collect(), garbage_colector.get_threshold()

[459, 354, 1608, 113, 888, 1534, 442, 945, 1301, 445, 255, 591, 99, 1212, 95, 1792, 1381, 671, 1332, 401, 1733, 1482, 294, 1587, 1790, 1157, 580, 172, 1345, 370, 44, 625, 109, 126, 335, 1349, 383, 530, 129, 137, 1185, 434, 168, 397, 269, 1323, 1364, 84, 1617, 155, 1221, 303, 1327, 1138, 218, 1675, 1027, 926, 896, 733, 670, 1473, 619, 523, 1569, 1605, 1606, 321, 1501, 93, 469, 1431, 164, 441, 800, 767, 1039, 1289, 213, 423, 1060, 1751, 1588, 1117, 1121, 1715, 182, 549, 1272, 390, 1154, 121, 1247, 560, 413, 568, 475, 475, 805, 1292, 489, 29, 989, 104, 71, 1486, 15, 581, 1086, 415, 585, 410, 1587, 29, 699, 1100, 1042, 1063, 872, 1685, 1033, 1501, 22, 968, 373, 344, 762, 262, 1098, 1696, 128, 773, 686, 157, 576, 58, 860, 1119, 941, 460, 1150, 1054, 1032, 299, 469, 31, 1114, 689, 1579, 1788, 1356, 1527, 1328, 639, 88, 1563, 1161, 794, 370, 365, 885, 1740, 1, 623, 338, 954, 605, 983, 406, 1375, 342, 596, 459, 1348, 1709, 1781, 725, 1063, 1178, 1120, 1191, 1628, 1002, 276, 1739, 1122, 776, 49

(0, (700, 10, 10))

## <center> **PROCESS POOL EXECUTOR DE CONCURRENT.FUTURES**</center>

In [55]:
%%time

with ProcessPoolExecutor() as executor:
    pool = executor.map(dormir_sincrono_multiprocessing, range(limite_tareas//3))
print(list(pool))

del executor, pool
garbage_colector.collect(), garbage_colector.get_threshold()

[227, 1396, 204, 1464, 916, 59, 1593, 1464, 932, 1707, 243, 665, 371, 1087, 1789, 146, 196, 1512, 507, 1108, 920, 1773, 1781, 713, 231, 1453, 1766, 603, 38, 347, 641, 160, 1183, 1651, 609, 165, 61, 183, 1692, 510, 1129, 1406, 437, 1775, 12, 11, 375, 1347, 99, 272, 549, 867, 31, 480, 1247, 14, 1164, 514, 1721, 1470, 535, 1105, 1255, 198, 43, 537, 681, 175, 757, 947, 1282, 1629, 869, 1534, 1413, 587, 1028, 1519, 1323, 832, 1182, 1754, 1398, 923, 262, 1770, 1626, 667, 1137, 42, 956, 1172, 1001, 1510, 1591, 1696, 1674, 908, 363, 369, 1235, 1481, 892, 1542, 696, 1058, 136, 354, 1349, 1395, 646, 1364, 917, 1367, 808, 901, 1686, 536, 1790, 461, 831, 375, 383, 1768, 87, 1032, 1179, 174, 1297, 289, 331, 813, 1481, 667, 1479, 795, 413, 157, 1671, 744, 1734, 779, 285, 1760, 1680, 680, 1374, 1639, 1555, 219, 785, 1527, 984, 116, 721, 785, 286, 695, 491, 251, 955, 100, 461, 1625, 1516, 1192, 561, 16, 1731, 619, 975, 443, 920, 373, 1729, 240, 319, 1680, 1252, 339, 500, 637, 1690, 1243, 56, 742, 1738

(0, (700, 10, 10))

## <center> **THREAD POOL EXECUTOR DE CONCURRENT.FUTURES**</center>

In [56]:
%%time

with ThreadPoolExecutor() as executor:
    pool = executor.map(dormir_sincrono_multiprocessing, range(limite_tareas//3))
print(list(pool))

del executor, pool
garbage_colector.collect(), garbage_colector.get_threshold()

[536, 1411, 170, 1664, 891, 1723, 486, 1302, 100, 864, 665, 511, 1087, 1565, 808, 220, 31, 1314, 1623, 1693, 1589, 635, 477, 1406, 1382, 984, 1442, 1165, 302, 911, 1648, 706, 921, 1765, 645, 333, 397, 341, 1330, 208, 159, 979, 739, 219, 840, 1210, 453, 1155, 1144, 796, 1763, 1776, 470, 1260, 1360, 1141, 1664, 513, 1262, 581, 1327, 1355, 1397, 1215, 644, 1076, 1304, 1420, 1630, 1793, 557, 248, 956, 164, 1761, 1778, 1716, 1107, 1576, 1423, 1569, 1631, 448, 1594, 152, 600, 262, 401, 640, 996, 823, 1499, 555, 634, 1371, 1293, 1473, 869, 1095, 1410, 963, 138, 547, 279, 444, 1523, 178, 786, 509, 977, 1126, 1652, 299, 908, 752, 14, 1707, 76, 863, 1536, 569, 258, 864, 1642, 1710, 885, 1690, 3, 294, 161, 1574, 1790, 969, 565, 888, 551, 681, 448, 434, 1541, 1610, 746, 71, 1533, 1571, 608, 1443, 1617, 1580, 1214, 260, 731, 829, 306, 1505, 1645, 1170, 1615, 1697, 1147, 457, 1403, 599, 1250, 1294, 230, 1342, 191, 1009, 451, 201, 1736, 593, 1191, 1562, 628, 1384, 1189, 1688, 2, 1259, 650, 74, 837, 1

(0, (700, 10, 10))

#  &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
#  &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
#  &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&

## <center> **COMPARTIR DATOS ENTRE HILOS Y PROCESOS**</center>

## <center> Compartir Datos Globales entre Hilos. 

In [57]:
%%time

compartir = 0
limite_hilos = 4
limite_iterador = 25

def memoria_compartida():
    
    global compartir    
    
    compartir_local  = compartir    
    
    for i in range(limite_iterador):
        compartir_local += 1
        
    sleep(0.2)
    
    for i in range(limite_iterador):
        compartir_local += 1
       
    compartir  = compartir_local
    
    print(f'Valor parcial {compartir}')

hilos = []

for hilo_numero in range(limite_hilos):
    
    hilo = Thread(target=memoria_compartida)
    hilo.start() 
    
    hilos.append(hilo)
    
for instancia_hilo in hilos:
    
    instancia_hilo.join()
    

print(f'Valor final {compartir}')

del compartir, limite_hilos, limite_iterador
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 50
Valor parcial 50
Valor parcial 50
Valor parcial 50
Valor final 50
CPU times: user 44.6 ms, sys: 154 µs, total: 44.8 ms
Wall time: 241 ms


(0, (700, 10, 10))

## <center> Definiendo la seccion critica. 

In [58]:
%%time

from threading import Lock
# instancia que bloquea hasta que finalice la seccion critica
seccion_critica = Lock()

compartir = 0
limite_hilos = 4
limite_iterador = 25

def memoria_compartida():
    
    global compartir    
    
    # debe estar ANTES de asignar el valor de la variable, que es el INICIO de la seccion critica
    seccion_critica.acquire()
    
    compartir_local  = compartir    
    
    for i in range(limite_iterador):
        compartir_local += 1
        
    sleep(0.2)
    
    for i in range(limite_iterador):
        compartir_local += 1
       
    compartir  = compartir_local
    
    # debe estar DESPUES de asignar el valor de la variable, que es el FINAL de la seccion critica
    seccion_critica.release()
    
    print(f'Valor parcial {compartir}')

hilos = []

for hilo_numero in range(limite_hilos):
    
    hilo = Thread(target=memoria_compartida)
    hilo.start()
    
    hilos.append(hilo)
    
for instancia_hilo in hilos:
    
    instancia_hilo.join()
    

print(f'Valor final {compartir}')

del seccion_critica, compartir, limite_hilos, limite_iterador
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 50
Valor parcial 100
Valor parcial 150
Valor parcial 200
Valor final 200
CPU times: user 47.2 ms, sys: 0 ns, total: 47.2 ms
Wall time: 842 ms


(0, (700, 10, 10))

## <center> Compartir Datos entre Procesos. 

In [59]:
%%time

compartir = 0
limite_procesos = 4
limite_iterador = 25
    
def datos_compartidos():
    
    global compartir
    
    compartir_local  = compartir
    
    for i in range(limite_iterador):
        compartir_local += 1
        
    sleep(0.2)
    
    for i in range(limite_iterador):
        compartir_local += 1
     
    compartir  = compartir_local
    print(f'Valor parcial {compartir}')

procesos = []

for hilo_numero in range(limite_procesos):
    
    proceso = Process(target=datos_compartidos)
    proceso.start()
    
    procesos.append(proceso)
    
for instancia_proceso in procesos:
    
    instancia_proceso.join()
    

print(f'Valor final {compartir}')

del compartir, limite_procesos, limite_iterador
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 50
Valor parcial 50
Valor parcial 50
Valor parcial 50
Valor final 0
CPU times: user 56.4 ms, sys: 63.7 ms, total: 120 ms
Wall time: 309 ms


(0, (700, 10, 10))

## <center> Sin definir la seccion critica. 

In [60]:
%%time
from multiprocessing import Value

# instancia que permite compartir los valores, definiendo el tipo de dato int
compartir = Value('i',0)

limite_procesos = 4
limite_iterador = 25

def datos_compartidos(): 
    
    global compartir
    
    for i in range(limite_iterador):
        compartir.value += 1
        
    sleep(0.2)
    
    for i in range(limite_iterador):
        compartir.value += 1
    
    print(f'Valor parcial {compartir.value}')

procesos = []

for hilo_numero in range(limite_procesos):
    
    proceso = Process(target=datos_compartidos)
    proceso.start()
    
    procesos.append(proceso)
    
for instancia_proceso in procesos:
    
    instancia_proceso.join()
    

print(f'Valor final {compartir.value}')

del compartir, limite_procesos, limite_iterador
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 125
Valor parcial 150
Valor parcial 175
Valor parcial 200
Valor final 200
CPU times: user 119 ms, sys: 23.5 ms, total: 142 ms
Wall time: 335 ms


(0, (700, 10, 10))

## <center> Definiendo la seccion critica. 

In [61]:
%%time
from multiprocessing import Lock

# instancia que bloquea hasta que finalice la seccion critica
seccion_critica = Lock()

# instancia que permite compartir los valores
compartir = Value('i',0)

limite_procesos = 4
limite_iterador = 25

def datos_compartidos():
    
    global compartir
    
    # debe estar ANTES de asignar el valor de la variable, que es el INICIO de la seccion critica
    seccion_critica.acquire() 
    
    for i in range(limite_iterador):
        compartir.value += 1
        
    sleep(0.2)
    
    for i in range(limite_iterador):
        compartir.value += 1
    
    # debe estar DESPUES de asignar el valor de la variable, que es el FINAL de la seccion critica
    seccion_critica.release()
    
    print(f'Valor parcial {compartir.value}')

procesos = []

for hilo_numero in range(limite_procesos):
    
    proceso = Process(target=datos_compartidos)
    proceso.start()
    
    procesos.append(proceso)
    
for instancia_proceso in procesos:
    
    instancia_proceso.join()
    

print(f'Valor final {compartir.value}')

del seccion_critica, compartir, limite_procesos, limite_iterador
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 50
Valor parcial 100
Valor parcial 150
Valor parcial 200
Valor final 200
CPU times: user 55.4 ms, sys: 50.8 ms, total: 106 ms
Wall time: 880 ms


(0, (700, 10, 10))

## <center> Compartir Datos entre THREAD POOL. 
### <center> SIN SECCION CRITICA. 

In [62]:
# instancia que permite compartir los valores
compartir = Value('i',0)
seccion_critica = Lock()

def datos_compartidos(incremento): 
    
    compartir.value += incremento
    sleep(0.2)    
    print(f'Valor parcial {compartir.value}')
    
    return compartir.value
 
with ThreadPool() as pool_de_hilos:
    multihilo = pool_de_hilos.map(datos_compartidos, range(1, LIMITE))

print(multihilo)
del multihilo, pool_de_hilos, compartir, seccion_critica
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 10Valor parcial 10

Valor parcial 21
Valor parcial 28
Valor parcial 36
Valor parcial 45
Valor parcial 55
Valor parcial 55
Valor parcial 55
Valor parcial 55
[28, 15, 21, 10, 55, 55, 36, 45, 55, 55]


(0, (700, 10, 10))

### <center> CON SECCION CRITICA. 

In [63]:
# instancia que permite compartir los valores
compartir = Value('i',0)
seccion_critica = Lock()

def datos_compartidos(incremento): 
    
    seccion_critica.acquire()
    compartir.value += incremento
    sleep(0.2)
    
    print(f'Valor parcial {compartir.value}') # acceder al valor antes de retornarlo puede interferir en el valor final retornado
    seccion_critica.release()   
    
    
    return compartir.value
    
with ThreadPool() as pool_de_hilos:
    multihilo = pool_de_hilos.map(datos_compartidos, range(1, LIMITE))

print(multihilo)
del multihilo, pool_de_hilos, compartir, seccion_critica
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 1
Valor parcial 3
Valor parcial 6
Valor parcial 10
Valor parcial 15
Valor parcial 21
Valor parcial 28
Valor parcial 36
Valor parcial 45
Valor parcial 55
[1, 3, 6, 10, 15, 21, 28, 36, 45, 55]


(0, (700, 10, 10))

## <center> Compartir Datos entre POOL de Procesos. 
## <center> Sin seccion critica. 

In [64]:
%%time
# instancia que permite compartir los valores
compartir = Value('i',0)
seccion_critica = Lock()

def datos_compartidos(incremento):
    
    global compartir
    
    compartir.value += incremento
    sleep(0.2)
    
    print(f'Valor parcial {compartir.value}')
    
    return compartir.value
    
with Pool() as pool_de_procesos:
    multiproceso = pool_de_procesos.map(datos_compartidos, range(1, LIMITE))
    
print(multiproceso)

del multiproceso, pool_de_procesos, compartir, seccion_critica
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 10Valor parcial 10Valor parcial 10Valor parcial 10



Valor parcial 36Valor parcial 36Valor parcial 36
Valor parcial 36


Valor parcial 55Valor parcial 55

[15, 10, 21, 28, 36, 45, 55, 55, 55, 55]
CPU times: user 56.7 ms, sys: 72.4 ms, total: 129 ms
Wall time: 727 ms


(0, (700, 10, 10))

## <center> Definiendo la seccion critica. 

In [65]:
# instancia que permite compartir los valores
compartir = Value('i',0)
seccion_critica = Lock()

def datos_compartidos(incremento):
    
    global compartir
    
    seccion_critica.acquire()
    
    compartir.value += incremento
    sleep(0.2)
    print(f'Valor parcial {compartir.value}')
    seccion_critica.release()  
      
    return compartir.value
    
    
with Pool() as pool_de_procesos:
    multiproceso = pool_de_procesos.map(datos_compartidos, range(1, LIMITE))
    
print(multiproceso)
del multiproceso, pool_de_procesos, compartir, seccion_critica
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 1
Valor parcial 3
Valor parcial 6
Valor parcial 10
Valor parcial 18
Valor parcial 24
Valor parcial 31
Valor parcial 36
Valor parcial 45
Valor parcial 55
[1, 6, 6, 10, 36, 24, 31, 18, 45, 55]


(0, (700, 10, 10))

## <center> Definiendo la seccion critica. **SIN PRINT**, OBTIENE LOS VALORES DE RETORNO CORRECTAMENTE, POR NO ACCEDER A ELLOS

In [66]:
# instancia que permite compartir los valores
compartir = Value('i',0)
seccion_critica = Lock()

def datos_compartidos(incremento):
    
    global compartir
    
    seccion_critica.acquire()
    
    compartir.value += incremento
    sleep(0.2)
    seccion_critica.release()  
      
    return compartir.value
    
    
with Pool() as pool_de_procesos:
    multiproceso = pool_de_procesos.map(datos_compartidos, range(1, LIMITE))
    
print(multiproceso)
del multiproceso, pool_de_procesos, compartir, seccion_critica
garbage_colector.collect(), garbage_colector.get_threshold()

[1, 3, 6, 10, 15, 21, 28, 36, 45, 55]


(0, (700, 10, 10))

# <center> POOL DE PROCESOS E HILOS OTROS METODOS</center>
## <center>Ofrece un medio conveniente de paralelizar la ejecución de una función a través de múltiples valores de entrada, distribuyendo los datos de entrada a través de procesos (paralelismo de datos).</center>

In [67]:
from multiprocessing.pool import Pool

def calcular_cuadrado(x):
    return x*x

### <center> APPLY - PROCESOS</center>

In [68]:
%%time
pool = Pool()
pool_apply = pool.apply(calcular_cuadrado, [3])
pool.close() # Prevents any more tasks from being submitted to the pool. Once all the tasks have been completed the worker processes will exit.
pool.join() # Wait for the worker processes to exit. One must call close() or terminate() before using join().
pool.close() # Close the Process object, releasing all resources associated with it.
pool_apply

print(pool_apply)
del pool_apply, pool
garbage_colector.collect(), garbage_colector.get_threshold()

9
CPU times: user 45.3 ms, sys: 64.6 ms, total: 110 ms
Wall time: 109 ms


(0, (700, 10, 10))

### <center> APPLY - HILOS</center>

In [69]:
%%time
pool = ThreadPool()
pool_apply = pool.apply(calcular_cuadrado, [4])
pool.close() # Prevents any more tasks from being submitted to the pool. Once all the tasks have been completed the worker processes will exit.
pool.join() # Wait for the worker processes to exit. One must call close() or terminate() before using join().
pool.close() # Close the Process object, releasing all resources associated with it.
pool_apply

print(pool_apply)
del pool_apply, pool
garbage_colector.collect(), garbage_colector.get_threshold()

16
CPU times: user 55.7 ms, sys: 189 µs, total: 55.9 ms
Wall time: 52.6 ms


(0, (700, 10, 10))

### <center> APPLY_ASYNC</center>

In [70]:
%%time
with Pool() as pool:
    values = pool.apply_async(calcular_cuadrado, [5])

print(values.get())
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

25
CPU times: user 55.2 ms, sys: 42.1 ms, total: 97.2 ms
Wall time: 102 ms


(0, (700, 10, 10))

### <center> MAP_ASYNC</center>

In [71]:
%%time
with Pool() as pool:
    values = pool.map_async(calcular_cuadrado, [6])

print(values.get())
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

[36]
CPU times: user 49.5 ms, sys: 41.6 ms, total: 91 ms
Wall time: 94 ms


(0, (700, 10, 10))

### <center> MAP</center>

In [72]:
%%time
with Pool() as pool:
    values = pool.map(calcular_cuadrado, range(0,20,5))

print(values)
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

[0, 25, 100, 225]
CPU times: user 81.6 ms, sys: 20.2 ms, total: 102 ms
Wall time: 99 ms


(0, (700, 10, 10))

### <center> IMAP</center>

In [73]:
%%time
with Pool() as pool:
    values = pool.imap(calcular_cuadrado, range(0,20,4))

print(values)
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

CPU times: user 79.3 ms, sys: 21.9 ms, total: 101 ms
Wall time: 106 ms


(152, (700, 10, 10))

### <center> IMAP_UNORDERED</center>

In [74]:
%%time
with Pool() as pool:
    values = pool.imap_unordered(calcular_cuadrado, range(0,20,2))

print(values)
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

CPU times: user 71.4 ms, sys: 31.9 ms, total: 103 ms
Wall time: 107 ms


(152, (700, 10, 10))

### <center> STARMAP</center>

In [75]:
%%time
with Pool() as pool:
    values = pool.starmap(calcular_cuadrado, [[1],[2],[3],[4],[5]])

print(values)
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

[1, 4, 9, 16, 25]
CPU times: user 49.4 ms, sys: 53.3 ms, total: 103 ms
Wall time: 104 ms


(0, (700, 10, 10))

### <center> STARMAP_ASYNC</center>

In [76]:
%%time
with Pool() as pool:
    values = pool.starmap_async(calcular_cuadrado, [[1],[2],[3],[4],[5]])

print(values)
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

CPU times: user 73.2 ms, sys: 33 ms, total: 106 ms
Wall time: 107 ms


(153, (700, 10, 10))

# <center> OTRAS **FUNCIONES ASINCRONAS**</center>

In [77]:
async def Primero(limite=5):
    print("Inicio funcion Primero")
    await asyncio.sleep(2)
    print("Final funcion Primero")
    
    return [random.randrange(-100, 0) for x in range(limite)]

async def Ultimo(limite=5):
    print("Comenzar funcion ultimo")
    await asyncio.sleep(2)
    print("Terminar funcion ultimo")
    
    return [random.randrange(0, 100) for x in range(limite)]

# <center> **GATHER**</center>
## <center> **RECIBE UNA LISTA DE TAREAS Y RETORNA UNA LISTA DE RESULTADOS**</center>
## <center> **TIENE OPCIONES ESPECÍFICAS PARA EL MANEJO DE ERRORES Y CANCELACIONES.**</center>

In [78]:
# El siguiente ejemplo muestra cómo esperar a que se completen varias tareas asincrónicas.

from asyncio import gather

async def iniciar_gather():
    
    tareas = [Primero(), Ultimo(), Primero(), Ultimo(), Primero(), Ultimo(), Primero()]
    
    return await gather(*tareas)
    
await iniciar_gather()

Inicio funcion Primero
Comenzar funcion ultimo
Inicio funcion Primero
Comenzar funcion ultimo
Inicio funcion Primero
Comenzar funcion ultimo
Inicio funcion Primero
Final funcion Primero
Terminar funcion ultimo
Final funcion Primero
Terminar funcion ultimo
Final funcion Primero
Terminar funcion ultimo
Final funcion Primero


[[-15, -71, -78, -1, -76],
 [23, 54, 88, 65, 95],
 [-68, -98, -20, -15, -83],
 [49, 72, 79, 74, 25],
 [-73, -65, -38, -61, -25],
 [79, 47, 6, 17, 17],
 [-15, -26, -69, -44, -99]]

# <center> **WAIT_FOR**</center>
## <center> **PERMITE DEFINIR EL MAXIMO TIEMPO DE ESPERA DE UNA TAREA**</center>

In [79]:
# El siguiente ejemplo demuestra cómo podemos utilizar un tiempo de espera para evitar esperar indefinida a que finalice una tarea asincrónica.

from asyncio import wait_for

async def iniciar_wait_for():
    try:
        return await wait_for(Primero(), timeout=1)
    except asyncio.TimeoutError:
        print("la ejecucion supero el tiempo de espera!")
        

await iniciar_wait_for()

Inicio funcion Primero
la ejecucion supero el tiempo de espera!


# <center> **AS_COMPLETED**</center>
## <center> **ES SIMILIAR A GATHER, PERO RETORNA FUTUROS, LOS RESULTADOS SON RETORNADOS EN EL ORDEN QUE ESTAN LISTOS**</center>

In [80]:
# El siguiente ejemplo demuestra cómo as_complete, completará la primera tarea, seguida de la siguiente más rápida y la siguiente hasta que se completen todas las tareas.

from asyncio import as_completed

async def iniciar_as_completed():
    
    tareas = [Primero(), Ultimo(), Primero(), Ultimo(), Primero(), Ultimo(), Primero()]
    counter = 0
    
    for future in as_completed(tareas):
        n = "la tarea mas rapida" if counter == 0 else "la siguiente tarea mas rapida"
        counter += 1
        result = await future
        print(f"{n} obtuvo el resultado: {result}")

await iniciar_as_completed()

Inicio funcion Primero
Comenzar funcion ultimo
Comenzar funcion ultimo
Inicio funcion Primero
Inicio funcion Primero
Comenzar funcion ultimo
Inicio funcion Primero
Final funcion Primero
Terminar funcion ultimo
Terminar funcion ultimo
Final funcion Primero
Final funcion Primero
Terminar funcion ultimo
Final funcion Primero
la tarea mas rapida obtuvo el resultado: [-76, -70, -54, -57, -39]
la siguiente tarea mas rapida obtuvo el resultado: [77, 57, 14, 97, 15]
la siguiente tarea mas rapida obtuvo el resultado: [55, 82, 7, 42, 41]
la siguiente tarea mas rapida obtuvo el resultado: [-14, -83, -98, -10, -93]
la siguiente tarea mas rapida obtuvo el resultado: [-28, -86, -94, -77, -24]
la siguiente tarea mas rapida obtuvo el resultado: [9, 79, 68, 69, 84]
la siguiente tarea mas rapida obtuvo el resultado: [-36, -44, -63, -39, -33]


# <center> **CREATE_TASK**</center>

In [81]:
# El siguiente ejemplo demuestra cómo convertir una rutina en una tarea y programarla en el bucle de eventos.

from asyncio import create_task

async def iniciar_create_task():
    
    tarea = create_task(Primero())
    print(tarea)
    
    await asyncio.sleep(2)
    print("MAS PROCESOS!")
    
    await asyncio.sleep(3)
    print(sum(tarea))
    
    return tarea

await iniciar_create_task()

<Task pending name='Task-1835' coro=<Primero() running at /tmp/ipykernel_98/4294725927.py:1>>
Inicio funcion Primero
MAS PROCESOS!
Final funcion Primero
0


<Task finished name='Task-1835' coro=<Primero() done, defined at /tmp/ipykernel_98/4294725927.py:1> result=[-7, -81, -28, -47, -14]>

# <center> **POOLS**</center>
# <center> **GET_RUNNING_LOOP**</center>

## <center> **ASYNCIO Event Loop - MULTI HILO**</center>

In [82]:
# importa la libtreria
from asyncio import get_running_loop
# importa librerias para trabajar concurrencia
from concurrent.futures import ThreadPoolExecutor

def blocking_io(value = 23, otros = 10):
    # File operations (such as logging) can block the
    # event loop: run them in a thread pool.
    with open("/dev/urandom", "rb") as f:
        return f.read(value + otros)
    
async def asyncio_multi_hilo():
    
    loop = get_running_loop()
    pool = ThreadPoolExecutor()
    
    # tipo de pool, funcion, parametros
    return await loop.run_in_executor(pool, blocking_io, 50, 70)

await asyncio_multi_hilo()

b'\xb7y4\xe5R\xe8\xf1\x079\xef\xad\xa2J\xfb\x9e\xf0\xe4\x08\x8b\x8b\x89|\x91_?\x82\xde\xc3W\xdc3\xbaH\xee\xfd[~\x03\x857\xef\xbbX\x98\xdb,Q\x9a\x04 \x19\t\x9b\xd8\xb7dJH\xb1\xf6O,\xde\xdd\xe9J\x05e8\x82\x1c\xc7\xf7\xa3uze\t|\xc1\x8c\xf5\x0b,d\x99\x10\x86\x05_\xe5C\x80\x17\x89\xec\x10\x90\xaf\xf5Qf\xe1\xf0\xef\xdd\xd9\x11\xd6R|0\xab3\xcf\xdb&b/\xcf'